In [ ]:
import numpy as np
import pandas as pd

In [ ]:
def training_td(data, window, ae, ae_dims, epochs, batch) :

  ## windowing
  windowing = np.zeros(shape=(window))
  for i in range(0, data.shape[0] - (window -1)) :
    windowing = np.vstack([windowing, np.array(data.iloc[i:i+window]).reshape(window)])

  windowing = windowing[1:]

  ## 모델 학습
  model, encoder = ae(window, ae_dims)

  history = model.fit(windowing, windowing, epochs=epochs , batch_size=batch, shuffle=True, verbose = 0)
  history.history['loss'][-1]

  ## 복원
  pred = model.predict(windowing)

  ## latent space
  latent_space1 = encoder.predict(windowing)

  ## 복원오차
  RE_test = windowing - pred
  RE_test = RE_test*RE_test
  RE_df_test = pd.DataFrame(RE_test)

  return model, windowing, RE_df_test

In [ ]:
def training_fd(data, window, stride, ae, ae_dims, epochs, batch) :

  ## windowing
  windowing = np.zeros(shape=(window))
  for i in range(0, data.shape[0] - (window -1),stride) :
    windowing = np.vstack([windowing, np.array(data.iloc[i:i+window]).reshape(window)])

  windowing = windowing[1:]

  ## transfer
  f_data = []
  for i in range(len(windowing)) :
    fft = np.fft.fft(windowing[i]) / len(windowing[i])
    fft_magnitude = abs(fft)

    f = np.linspace(-(100 / 2), 100 / 2, len(windowing[i]))
    f_data.append(np.fft.fftshift(fft_magnitude).tolist())

  f_data = np.array(f_data)

  ## 모델 학습
  model, encoder = ae(window, ae_dims)

  history = model.fit(f_data, f_data, epochs=epochs , batch_size=batch, shuffle=True, verbose = 0)
  history.history['loss'][-1]

  ## 복원
  pred = model.predict(f_data)

  ## latent space
  latent_space1 = encoder.predict(f_data)

  ## 복원오차
  RE_test = f_data - pred
  RE_test = RE_test*RE_test
  RE_df_test = pd.DataFrame(RE_test)

  return model, windowing, f_data, RE_df_test

In [ ]:
def training(data, cycle, trend, window, ae, ae_dims, epoch1, batch1, epoch2, batch2, stride) :
  ############################################ Cycle
  model, windowing, RE_df_test = training_td(cycle, window, ae, ae_dims, epoch1, batch1)

  # 개별 복원오차
  tmp = np.zeros(shape=(window-1,window))
  tmp[:] = np.nan

  RE_lag_test = pd.concat([RE_df_test, pd.DataFrame(tmp)], ignore_index=True, axis=0)

  for i in range(1,window):
      RE_lag_test[i] = RE_lag_test[i].shift(i)


  RE_final_test_cycle = RE_lag_test.mean(axis=1)
  mean = RE_final_test_cycle.mean()
  std = RE_final_test_cycle.std(ddof=1)
  anomaly_idx_time = np.where((RE_final_test_cycle > mean + 3*std)|(RE_final_test_cycle < mean - 3*std))[0]

  ############################################ Trend
  model, windowing, f_data, RE_df_test = training_fd(trend, window, stride, ae, ae_dims, epoch2, batch2)

  # 개별 복원오차
  RE_df_test_t = pd.DataFrame([RE_df_test.mean(axis=1)]*window)
  tmp = np.zeros(shape=(len(trend)-len(RE_df_test_t),len(RE_df_test_t)))
  tmp[:] = np.nan

  RE_lag_test = pd.concat([RE_df_test_t, pd.DataFrame(tmp)], ignore_index=True, axis=0)

  for i in range(1,RE_lag_test.shape[1]):
      RE_lag_test.iloc[:,i] = RE_lag_test.iloc[:,i].shift(i*stride)

  RE_final_test_trend = RE_lag_test.mean(axis=1)
  mean = RE_final_test_trend.mean()
  std = RE_final_test_trend.std(ddof=1)
  anomaly_idx_freq = np.where((RE_final_test_trend > mean + 3*std)|(RE_final_test_trend < mean - 3*std))[0]

  ############################################ Total
  data['anomaly_idx_2domain'] = 0
  anomaly_idx_2domain = list(anomaly_idx_freq) + list(anomaly_idx_time)
  data.iloc[anomaly_idx_2domain, 2] = 1

  return data, RE_final_test_cycle, RE_final_test_trend, anomaly_idx_2domain